<h1>AWAC Validation Figures</h1>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy import signal
from scipy import interpolate
from scipy import optimize
from scipy import io
import cftime
import netCDF4 as nc
import pickle
import glob
import scipy.stats as st
%matplotlib widget

In [2]:
from ahrs.filters import EKF
from scipy.spatial.transform import Rotation as R

def ekfCorrection(accel_x, accel_y, accel_z, gyro_x, gyro_y, gyro_z, mag_x, mag_y, mag_z):
    '''
    @edwinrainville

    Correct the body frame accelerations to the earth frame of reference through and extended Kalman Filter
    '''

    # Organize the acceleration data into arrays to be used in the ekf algorithm
    acc_data = np.array([accel_x-np.mean(accel_x), accel_y-np.mean(accel_y), accel_z-np.mean(accel_z)]).transpose()
    gyr_data = np.array([gyro_x-np.mean(gyro_x), gyro_y-np.mean(gyro_y), gyro_z-np.mean(gyro_z)]).transpose()
    mag_data = np.array([mag_x-np.mean(mag_x), mag_y-np.mean(mag_y), mag_z-np.mean(mag_z)]).transpose()

    # Rotate the acceleration data with extended Kalman filter
    # ** the variance of each sensor needs to be further examined
    # ** Variance is assumed from spec sheet
    # ekf = EKF(gyr=gyr_data, acc=acc_data, mag_data=mag_data, frequency=12, var_acc=0.000003, var_gyro=0.04, var_mag=.1, frame='NED')
    ekf = EKF(gyr=gyr_data, acc=acc_data, frequency=12, var_acc=0.000003, var_gyro=0.04, frame='NED')

    # Rotate the acclerations from the computed Quaterions
    r = R.from_quat(ekf.Q)
    accel_rotated = r.apply(acc_data)

    # Get acceleration data from the rotated structure
    accel_x_earth = accel_rotated[:,0]
    accel_y_earth = accel_rotated[:,1]
    accel_z_earth = accel_rotated[:,2]

    return accel_x_earth, accel_y_earth, accel_z_earth 

def computeEta(accel_z, fs, low_freq_cutoff, high_freq_cutoff, order):       
    # Define the filter
    b, a = signal.butter(order, [low_freq_cutoff, high_freq_cutoff], btype='bandpass', fs=fs)
    # b, a = signal.butter(order, low_freq_cutoff, btype='highpass', fs=fs)

    # Zero pad the edges to reduce edge effects
    pad_size = 500
    a_z_padded = np.zeros(accel_z.size + pad_size*2)
    a_z_padded[pad_size:-pad_size] = accel_z

    # Filter and integrate to velocity and position
    a_z = signal.filtfilt(b, a, a_z_padded)
    w_nofilt = integrate.cumulative_trapezoid(a_z, dx=1/fs, initial=0)
    w = signal.filtfilt(b, a, w_nofilt)
    z_nofilt = integrate.cumulative_trapezoid(w, dx=1/fs, initial=0)
    z = signal.filtfilt(b, a, z_nofilt)

    # Remove the edges of the time series due to the edge effects of filtering
    edge_removed = 100
    # t = t[edge_removed : -edge_removed]
    a_z = a_z[pad_size + edge_removed : -(pad_size + edge_removed)]
    w = w[pad_size + edge_removed : -(pad_size + edge_removed)]
    z = z[pad_size + edge_removed : -(pad_size + edge_removed)]

    return a_z, w, z

def computeSpectra(z, fs):
    nperseg = 3600
    overlap = 0.50
    f_raw, E_raw = signal.welch(z, fs=fs, window='hann', nperseg=nperseg, noverlap=np.floor(nperseg*overlap))

    # Band Average the Spectra
    points_to_average = 5
    num_sections = E_raw.size // points_to_average
    f_chunks = np.array_split(f_raw, num_sections)
    E_chunks = np.array_split(E_raw, num_sections)
    f = np.array([np.mean(chunk) for chunk in f_chunks ])
    E = np.array([np.mean(chunk) for chunk in E_chunks ])
    # dof = np.floor(2 * z.shape[0]//nperseg) * points_to_average
    dof = np.floor(points_to_average * (8/3) * (z.shape[0]/ (nperseg//2)))
    return f, E, dof

def processZAccel(accel_z, fs, low_freq_cutoff, high_freq_cutoff, order):
    a_z, w, z = computeEta(accel_z, fs, low_freq_cutoff, high_freq_cutoff, order)

    # Compute the spectra
    f, E, dof = computeSpectra(z, fs)

    return a_z, w, z, f, E, dof

def computeBulkWaveParameters(f, E):
    # Compute Significant wave height 
    f_inds = np.where((f >= 0.04) & (f <= 0.5))[0]
    f_waves = np.array([f[i] for i in f_inds])
    E_waves = np.array([E[i] for i in f_inds])
    E_integrated = integrate.trapezoid(E_waves, f_waves)
    Hs = 4 * np.sqrt(E_integrated)
    
    # Compute Peak Period
    Tp = 1 / np.squeeze(f_waves[np.where(E_waves == np.amax(E_waves))])

    return Hs, Tp

def avgDistToAWAC(x, y, x_awac, y_awac):
    '''
    @edwinrainville

    Compute the distance from the 4.5 m AWAC to the average location of the microSWIFT
    '''
    # Compute average location of microSWIFT during track 
    x_avg = np.mean(x)
    y_avg = np.mean(y)

    # Compute Distance from AWAC to average location of microSWIFT
    dist = np.sqrt((x_avg - x_awac)**2 + (y_avg - y_awac)**2)

    return dist

def transform2FRF(lat,lon):
    '''
    @edwinrainville, Originally written by J. Thomson, 1/2011

    Description: function to convert from lat & lon (decimal degrees, negative longitude) to FRF x,y (meters)
    '''

    # Define offsets
    lat_offset = 36.178039
    lon_offset = -75.749672

    # Define constants
    rotation = 19 #rotation in degress CCW from True north

    # Radius of Earth
    earth_rad = 6378.1 * 1000 # units are meters

    # correct radius for latitutde 
    radius_at_latoffset = earth_rad * np.cos(np.deg2rad(np.median(lat_offset))) 

    # Compute North-South and East-West Locations
    north = earth_rad * np.deg2rad(lat- lat_offset)
    east = radius_at_latoffset * np.deg2rad(lon_offset - lon) 

    # Rotate Coordinates by 19 degrees CCW from True north
    x = east * np.cos(np.deg2rad(rotation))   -   north * np.sin (np.deg2rad(rotation))
    x = -x # Flip x 
    y = east * np.sin(np.deg2rad(rotation))   +   north * np.cos (np.deg2rad(rotation))

    # return x and y values
    return x, y

In [3]:
# Data from 4.5 meter AWAC
awac_data = nc.Dataset('../microSWIFT_data/FRFdata/FRF-ocean_waves_awac-4.5m_202110.nc')
xFRF_awac, yFRF_awac = transform2FRF(np.float64(awac_data['latitude'][:]), np.float64(awac_data['longitude'][:]))

# AWAC Confidence interval
from scipy.stats import chi2
probability = 0.95
alpha = 1 - probability

# AWAC confidence interval
DOF_awac = 48
c_awac = chi2.ppf([1 - alpha / 2, alpha / 2], DOF_awac)
c2_awac = DOF_awac / c_awac

# Filtering Characteristics
low_freq_cutoff = 0.05
high_freq_cutoff = 2
filt_order = 1
fs = 12.0